In [1]:
import pandas
import pathlib
from config import datadir
import numpy
import matplotlib.pyplot as plt

In [2]:
datadir

WindowsPath('C:/Users/jarry/Google Drive/University/CSC411/CSC 2020 Webber J/Data')

In [3]:
excel_directory = pandas.read_excel(datadir/'Data Description.xlsx', parse_dates=True, index_col='Start Date')
excel_directory

,End Date,Equipment,Type,Purpose,Filename,Run,Description
Start Date,,,,,,,
2019-12-06,2019-12-11,pH Probe,Drift,Test Drift on pH probe,20191206 pH Drift 1.txt,1,"long run of pH probe drift, performed inside t..."
2019-12-11,2019-12-17,pH Probe,Drift,Test Drift on pH probe,20191211 pH Drift 2.txt,2,"long run of pH probe Drift, circulating ph7 bu..."
2019-12-17,2019-12-23,pH Probe,Drift,Test Drift on pH probe,20191217 pH Drift 3.txt,3,"long run of pH probe drift, performed inside t..."
2020-01-16,2020-01-16,pH Probe,Step Test,NaN,20200116 pH Step 3.txt,3,pH stepped outside of rig
2020-01-14,2020-01-14,pH Probe,Step Test,NaN,20200116 pH Step 1.txt,1,"pH stepped outside of rig. Probe Broke, Dump run"
2020-01-15,2020-01-15,pH Probe,Step Test,NaN,20200116 pH Step 2.txt,2,pH stepped outside of rig


In [76]:
drifts = excel_directory[excel_directory['Type']=='Drift']
drifts

,End Date,Equipment,Type,Purpose,Filename,Run,Description
Start Date,,,,,,,
2019-12-06,2019-12-11,pH Probe,Drift,Test Drift on pH probe,20191206 pH Drift 1.txt,1,"long run of pH probe drift, performed inside t..."
2019-12-11,2019-12-17,pH Probe,Drift,Test Drift on pH probe,20191211 pH Drift 2.txt,2,"long run of pH probe Drift, circulating ph7 bu..."
2019-12-17,2019-12-23,pH Probe,Drift,Test Drift on pH probe,20191217 pH Drift 3.txt,3,"long run of pH probe drift, performed inside t..."


In [70]:
all_drifts_frames =[]
durations = []
for file in drifts['Filename']:
    new_frame = pandas.read_csv(datadir /file, 
                               sep ='\t',header = None,
                               usecols = [0, 1, 2, 3],
                               names = ["Date", "Time", "Temp", "pH"],
                               parse_dates = [['Date', 'Time']],
                               infer_datetime_format = True,
                               index_col='Date_Time')
    
    durations.append([new_frame.index[0],new_frame.index[-1]])
    all_drifts_frames.append(new_frame)
        
    print (datadir /file)

C:\Users\jarry\Google Drive\University\CSC411\CSC 2020 Webber J\Data\20191206 pH Drift 1.txt
C:\Users\jarry\Google Drive\University\CSC411\CSC 2020 Webber J\Data\20191211 pH Drift 2.txt
C:\Users\jarry\Google Drive\University\CSC411\CSC 2020 Webber J\Data\20191217 pH Drift 3.txt


In [79]:
durations, all_drifts_frames[0]['2019-12-06 10:00:00':'2019-12-06 12:00:00']

([[Timestamp('2019-12-06 09:05:27'), Timestamp('2019-12-11 11:48:00')],
  [Timestamp('2019-12-11 12:13:30'), Timestamp('2019-12-17 10:01:59')],
  [Timestamp('2019-12-17 10:59:24'), Timestamp('2019-12-23 09:52:57')]],
                           Temp        pH
 Date_Time                               
 2019-12-06 10:00:00  34.557765  6.741578
 2019-12-06 10:00:01  34.558060  6.660625
 2019-12-06 10:00:02  34.558414  6.623300
 2019-12-06 10:00:03  34.558246  6.646065
 2019-12-06 10:00:04  34.557381  6.714695
 ...                        ...       ...
 2019-12-06 11:59:56  36.167234  6.380995
 2019-12-06 11:59:57  36.167991  6.355328
 2019-12-06 11:59:58  36.168512  6.439562
 2019-12-06 11:59:59  36.167745  6.520760
 2019-12-06 12:00:00  36.167490  6.494997
 
 [7201 rows x 2 columns])

In [75]:
for frame in all_drifts_frames:
    print(frame.index[0],frame.index[-1])

2019-12-06 09:05:27 2019-12-11 11:48:00
2019-12-11 12:13:30 2019-12-17 10:01:59
2019-12-17 10:59:24 2019-12-23 09:52:57
